In [80]:
import warnings
warnings.filterwarnings("ignore")

In [81]:
import pandas as pd

df = pd.read_csv("../input/news_summary.csv", encoding="latin-1")
df.head(3)

,author,date,headlines,read_more,text,ctext
0,Chhavi Tyagi,"03 Aug 2017,Thursday",Daman & Diu revokes mandatory Rakshabandhan in...,http://www.hindustantimes.com/india-news/raksh...,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...
1,Daisy Mowke,"03 Aug 2017,Thursday",Malaika slams user who trolled her for 'divorc...,http://www.hindustantimes.com/bollywood/malaik...,Malaika Arora slammed an Instagram user who tr...,"From her special numbers to TV?appearances, Bo..."
2,Arshiya Chopra,"03 Aug 2017,Thursday",'Virgin' now corrected to 'Unmarried' in IGIMS...,http://www.hindustantimes.com/patna/bihar-igim...,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...


In [82]:
df = df[["text", "ctext"]]
df.ctext = "summarize: " + df.ctext
print(df.shape)
df.head(3)

(4514, 2)


,text,ctext
0,The Administration of Union Territory Daman an...,summarize: The Daman and Diu administration on...
1,Malaika Arora slammed an Instagram user who tr...,summarize: From her special numbers to TV?appe...
2,The Indira Gandhi Institute of Medical Science...,summarize: The Indira Gandhi Institute of Medi...


In [41]:
text = str(df.text[0])
text = " ".join(text.split())

ctext = str(df.ctext[0])
ctext = " ".join(ctext.split())


In [42]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")

In [66]:
source = tokenizer.batch_encode_plus(
    [ctext], 
    max_length=512, 
    pad_to_max_length=True,
    return_tensors="pt",
    truncation=True,
)

print(source)

{'input_ids': tensor([[21603,    10,    37,   878,   348,    11,  2043,    76,  3602,    30,
          2875,    28,    26,    60,   210,     3,     9, 15646,    24,  1380,
           887,   871,    12,  6177,     3,  9782, 10193,    30,  5069,  6976,
           227,     8,   455,     3, 18728,     3,     9,   223,  8058,    45,
          1652,    11,    47,     3, 23830,  3943,    30,   569,   783,     5,
           634,  7021,  9964,    58,     7,  3602,    47,  5241,    12, 10742,
           441,   997,   716,    13,    19, 17180,     8, 15646,    24,   263,
            34, 29701,    21,   165,   871,    12,  4036,  2922,   157,     7,
          6111,   232,  2618,    44,  6940,     5,    58,   196,    17,    65,
           118,  1500,    12,  4036,     8,  3994,    13,  2922,   157,     7,
          6111,   232,  2618,    30,  1660,  4306,    86,    48,  2135,     6,
            66,  6036,    87, 10521,  1522,  2367,   539,    11,  4036,     8,
          3994,  6018,   120,    44,  

In [67]:
source_ids = source["input_ids"].squeeze()

print(type(source_ids))
print(source_ids.shape)
print(source_ids)
print("-"*10)

source_mask = source["attention_mask"].squeeze()

print(type(source_mask))
print(source_mask.shape)
print(source_mask)


<class 'torch.Tensor'>
torch.Size([512])
tensor([21603,    10,    37,   878,   348,    11,  2043,    76,  3602,    30,
         2875,    28,    26,    60,   210,     3,     9, 15646,    24,  1380,
          887,   871,    12,  6177,     3,  9782, 10193,    30,  5069,  6976,
          227,     8,   455,     3, 18728,     3,     9,   223,  8058,    45,
         1652,    11,    47,     3, 23830,  3943,    30,   569,   783,     5,
          634,  7021,  9964,    58,     7,  3602,    47,  5241,    12, 10742,
          441,   997,   716,    13,    19, 17180,     8, 15646,    24,   263,
           34, 29701,    21,   165,   871,    12,  4036,  2922,   157,     7,
         6111,   232,  2618,    44,  6940,     5,    58,   196,    17,    65,
          118,  1500,    12,  4036,     8,  3994,    13,  2922,   157,     7,
         6111,   232,  2618,    30,  1660,  4306,    86,    48,  2135,     6,
           66,  6036,    87, 10521,  1522,  2367,   539,    11,  4036,     8,
         3994,  6018,  

In [68]:
target = tokenizer.batch_encode_plus(
    [text], 
    max_length=150, 
    pad_to_max_length=True,
    return_tensors="pt",
    truncation=True,
)

print(target)

{'input_ids': tensor([[   37,  6863,    13,  3545, 30110,   878,   348,    11,  2043,    76,
            65,     3,    52, 17943,    26,   165,   455,    24,   263,    34,
         29701,    21,   887,    12,  6177,     3,  9782, 10193,    12,    70,
          5069,  6976,    30,     8,  5333,    13,  2922,   157,     7,  6111,
           232,  2618,    30,  1660,  4306,    37,  3602,    47,  5241,    12,
         14510,     8,  1357,   441,   997,   716,    13,    19, 17180,     8,
         15646,   227,    34,  1204,  5731,   157,    45,  1652,    11,    47,
             3,     7,    40,   265,  2726,    30,   569,   783,     5,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [69]:
target_ids = target["input_ids"].squeeze()

print(type(target_ids))
print(target_ids.shape)
print(target_ids)
print("-"*10)

target_mask = target["attention_mask"].squeeze()

print(type(target_mask))
print(target_mask.shape)
print(target_mask)


<class 'torch.Tensor'>
torch.Size([150])
tensor([   37,  6863,    13,  3545, 30110,   878,   348,    11,  2043,    76,
           65,     3,    52, 17943,    26,   165,   455,    24,   263,    34,
        29701,    21,   887,    12,  6177,     3,  9782, 10193,    12,    70,
         5069,  6976,    30,     8,  5333,    13,  2922,   157,     7,  6111,
          232,  2618,    30,  1660,  4306,    37,  3602,    47,  5241,    12,
        14510,     8,  1357,   441,   997,   716,    13,    19, 17180,     8,
        15646,   227,    34,  1204,  5731,   157,    45,  1652,    11,    47,
            3,     7,    40,   265,  2726,    30,   569,   783,     5,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,  

In [70]:
class NewsSummaryDataset():
    def __init__(self, dataframe):
        self.data = dataframe
        self.text = self.data.text
        self.ctext = self.data.ctext
    
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, index):
        return {
            "text": self.text, 
            "ctext": self.ctext
        }

dataset = NewsSummaryDataset(df)


In [71]:
len(dataset)

4514

In [72]:
dataset["text"]

{'text': 0       The Administration of Union Territory Daman an...
 1       Malaika Arora slammed an Instagram user who tr...
 2       The Indira Gandhi Institute of Medical Science...
 3       Lashkar-e-Taiba's Kashmir commander Abu Dujana...
 4       Hotels in Maharashtra will train their staff t...
                               ...                        
 4509    Fruit juice concentrate maker Rasna is eyeing ...
 4510    Former Indian cricketer Sachin Tendulkar atten...
 4511    Aamir Khan, while talking about reality shows ...
 4512    The Maharashtra government has initiated an in...
 4513    At least 400 languages or more than half langu...
 Name: text, Length: 4514, dtype: object,
 'ctext': 0       summarize: The Daman and Diu administration on...
 1       summarize: From her special numbers to TV?appe...
 2       summarize: The Indira Gandhi Institute of Medi...
 3       summarize: Lashkar-e-Taiba's Kashmir commander...
 4       summarize: Hotels in Mumbai and other Indian c.

In [87]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [88]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        dataframe, 
        tokenizer, 
        source_len, 
        summ_len
    ):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext
        
    def __len__(self):
        return len(self.text)
        
    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus(
            [ctext], 
            max_length=self.source_len, 
            pad_to_max_length=True,
            return_tensors="pt",
            truncation=True,
        )
        target = self.tokenizer.batch_encode_plus(
            [text], 
            max_length=self.summ_len, 
            pad_to_max_length=True,
            return_tensors="pt",
            truncation=True,
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long), 
            "source_mask": source_mask.to(dtype=torch.long), 
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long)
        }
    

In [89]:
TRAIN_BATCH_SIZE = 2
MAX_LEN = 512
SUMMARY_LEN = 150 

training_set = CustomDataset(
    df, 
    tokenizer, 
    MAX_LEN, 
    SUMMARY_LEN
)
    
train_params = {
    "batch_size": TRAIN_BATCH_SIZE,
    "shuffle": True,
    "num_workers": 0
}

training_loader = DataLoader(
    training_set, 
    **train_params
)

In [90]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
TRAIN_EPOCHS = 1
LEARNING_RATE = 1e-4

for epoch in range(TRAIN_EPOCHS):
    model.train()
    for _, data in enumerate(training_loader, 0):
        ids = data["source_ids"].to(device, dtype=torch.long)
        #print(ids)
    
        mask = data["source_mask"].to(device, dtype=torch.long)
        #print(mask)
    
        y = data["target_ids"].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        #print(y_ids.shape)
        #print(y_ids)
    
        lm_labels = y[:, 1:].clone().detach()
        #print(lm_labels)
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        #print(lm_labels)
    
        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            lm_labels=lm_labels
        )
    
        loss = outputs[0]
    
        if _ % 500 == 0:
            print(f"Epoch: {epoch}, Loss:  {loss.item()}")
    
    
        optimizer = torch.optim.Adam(
            params=model.parameters(), 
            lr=LEARNING_RATE
        )
    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
     

Epoch: 0, Loss:  7.992527008056641


KeyboardInterrupt: 